In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import keras
from keras import regularizers
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
import numpy as np
import seaborn as sns
import os
import re
import time
import pickle
sns.set()

Using TensorFlow backend.


In [3]:
def get_filenames(path='.', extension=None, pattern=None, identifiers=None, include_path=False):
   
    # retrieve all filenames from the directory
    filename_list = os.listdir(path)
    
    # keep all filenames with the proper extension
    if extension is not None:
        
        filename_list = [filename for filename in filename_list if
                         filename[-len(extension):] == extension]
        
    # keep all filenames that match the pattern
    if pattern is not None:
        filename_list = [filename for filename in filename_list if re.search(pattern, filename)]
        
    # keep all filenames that match the identifiers provided
    if identifiers is not None:
        storage_list = []
        
        try:
            for ident in identifiers:
                storage_list.extend([filename for filename in filename_list if str(ident) in filename])
                
        except TypeError:
            print(identifiers, 'is not a list, tuple, or otherwise iterable')
            
        else:
            filename_list = storage_list
            
    if include_path:
        filename_list = [path + filename for filename in filename_list]
        
    return filename_list


def get_filevalues(path, filename_list): 
    
    # empty lists 
    list_fluxarrays = []
    list_classtype = []
    list_noise = []
    list_wavelength = []
    list_redshift = []
    
    # going through all the fits files
    for i in range(len(filename_list)):
        with fits.open(str(path) +str(filename_list[i])+ "", memmap = False ) as hdul:
            
            data_c = hdul['COADD'].data 
           
            # the 2nd HDU is different in certain quasars
            # this is appending all the ones with "SPALL" as the their 2nd HDU
            if hdul[2].name == "SPALL":
                
                data_s = hdul['SPALL'].data
                
                flux_val = data_c.field("flux")
                list_fluxarrays.append(flux_val) 
            
                classtype = data_s.field('CLASS')
                list_classtype.append(classtype)
            
                noise_val = data_s.field('SN_MEDIAN_ALL')
                list_noise.append(noise_val)
            
                wavelength_val = data_c.field('loglam')
                list_wavelength.append(wavelength_val)
            
                redshift_val = data_s.field('Z')
                list_redshift.append(redshift_val)
                
                del hdul['SPALL'].data
            
            # this is appending all the values for the quasars with "SPECOBJ" as their 2nd HDU
            elif hdul[2].name == "SPECOBJ":
                
                data_s = hdul['SPECOBJ'].data
                
                flux_val = data_c.field("flux")
                list_fluxarrays.append(flux_val) 
            
                classtype = data_s.field('CLASS')
                list_classtype.append(classtype)
            
                noise_val = data_s.field('SN_MEDIAN_ALL')
                list_noise.append(noise_val)
            
                wavelength_val = data_c.field('loglam')
                list_wavelength.append(wavelength_val)
            
                redshift_val = data_s.field('Z')
                list_redshift.append(redshift_val)
                
                del hdul['SPECOBJ'].data
            
            else:
                pass
            
            
            values = {'FLUX': list_fluxarrays, 'CLASS': list_classtype, 'NOISE': list_noise,\
                      'WAVE': list_wavelength, 'REDSHIFT': list_redshift }
            
            hdul.close()
            del hdul['COADD'].data
            del hdul['PRIMARY'].data
            del hdul
            
    return values    

def save_data_to_disk(name_file, saved_variable):
    
    filename = str(name_file)
    outfile = open(filename,'wb')
    
    pickle.dump(saved_variable,outfile)
    outfile.close()

In [4]:
quasardata = get_filenames("/Users/matt/Desktop/DESI_Research/DESI_ML/Quasars_data/", extension='.fits')
quasar_dict = get_filevalues("/Users/matt/Desktop/DESI_Research/DESI_ML/Quasars_data/", quasardata)

In [5]:
stardata = get_filenames("/Users/matt/Desktop/DESI_Research/DESI_ML/Stars_data/", '.fits')
star_dict = get_filevalues("/Users/matt/Desktop/DESI_Research/DESI_ML/Stars_data/", stardata)

In [8]:
z = (star_dict['REDSHIFT'])

In [27]:
new_z = []
    
for i in range(len(z)):
    if (z[i] >= 2) & (z[i] <= 3.0):
        new_z.append(z[i])
print(len(new_z))

1158


In [ ]:
filename = 'quasar_data'
outfile = open(filename,'wb')

pickle.dump(quasar_dict,outfile)
outfile.close()

In [ ]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()

In [ ]:
#print(quasar_dict)
#print(new_dict==quasar_dict)
print(type(new_dict))

In [ ]:
len(quasar_dict['FLUX'])

In [ ]:
len(new_dict['FLUX'])

In [ ]:
filename = 'star_data'
outfile = open(filename,'wb')

pickle.dump(star_dict,outfile)
outfile.close()